<a href="https://colab.research.google.com/github/WillFuturi35/Alg-y-Estruc-de-datos-UPC/blob/main/ActividadArbol/ActividadArbol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
%%writefile ArbolB.hpp
#ifndef __ARBOLB_HPP__
#define __ARBOLB_HPP__

template <class T>
class Nodo {
public:
	T elemento;
	Nodo* izq;
	Nodo* der;
};

template <class T>
class ArbolB {
	Nodo<T>* raiz;
	void(*procesar)(T); //Puntero a funcion

private:
	bool _insertar(Nodo<T>*& nodo, T e) {
		if (nodo == nullptr) {
			nodo = new Nodo<T>();
			nodo->elemento = e;
		}
		else if (e < nodo->elemento) {
			return _insertar(nodo->izq, e);
		}
		else if (e >= nodo->elemento) {
			return _insertar(nodo->der, e);
		}
	}

	void _enOrden(Nodo<T>* nodo) {
		if (nodo == nullptr) return;
		_enOrden(nodo->izq);
		procesar(nodo->elemento);
		_enOrden(nodo->der);
	}

	void _preOrden(Nodo<T>* nodo) {
		if (nodo == nullptr) return;
		procesar(nodo->elemento);
		_preOrden(nodo->izq);
		_preOrden(nodo->der);
	}

	void _postOrden(Nodo<T>* nodo) {
		if (nodo == nullptr) return;
		_postOrden(nodo->izq);
		_postOrden(nodo->der);
		procesar(nodo->elemento);
	}

	bool _vacio() {
		return raiz == nullptr;
	}

	int _cantidad(Nodo<T>* nodo) {
		//La cantidad de nodos del árbol es:
		//	0 si es vacío
		//	1 + la cantidad de nodos por la izquierda + la cantidad de nodos por la derecha

		if (nodo==nullptr)
			return 0;
		else
		{
			int ci, cd;
			ci = _cantidad(nodo->izq);
			cd = _cantidad(nodo->der);
			return 1 + ci + cd;
		}

	}

	int _altura(Nodo<T>* nodo) {
		//La altura del árbol es:
		//	0 si es vacío
		//	la mayor de las alturas por la izquierda y por la derecha, las cuáles son 0 si son vacías ó 1 + la altura por la izq(o der) en caso contrario

		if (nodo==nullptr)
			return 0;
		else
		{
			int ai, ad;
			ai = 1 + _altura(nodo->izq);
			ad = 1 + _altura(nodo->der);
			return ai > ad ? ai : ad;			
		}
	}

    bool _eliminar(Nodo<T>* nodo,T tipo){
        if(nodo==nullptr)
            return 0;
        else{
            int raiz=comparar(nodo->elemento,tipo);
            if(raiz<0){
                return eliminar(nodo->der,tipo);
            }else if(raiz>0){
                return _eliminar(nodo->izq,tipo);
            }else{ //r==0 encontró el elemento e en el arbol
                if(nodo->der== nullptr,nodo->izq==nullptr){
                    nodo= nullptr;
                    _eliminar(nodo);
                    return true;
                }else if(nodo->izq==nullptr){
                    nodo=nodo->der;
                    return true;
                }else if(nodo->der==nullptr){
                    nodo=nodo->izq;
                    return true;
                }else{
                    Nodo<T>* aux=new Nodo<T>;
                    aux=aux->izq; //buscara el menor elemento por la derecha
                    while(aux->izq != nullptr){
                        aux=aux->izq;//Se obtiene la hoja menor
                    }
                    nodo->elemento=aux->elemento;//Se actualiza el elemento en el nodo raiz y
                    return _eliminar(nodo->der,aux->elemento);
                    //se envía a eliminar el elemento en el arbol por la derecha
                }
            }
        }
    }


public:
	ArbolB(void(*otroPunteroAFuncion)(T)) {
		this->procesar = otroPunteroAFuncion;
		raiz = nullptr;
	}
	bool insertar(T e) {
		return _insertar(raiz, e);
	}
	void enOrden() {
		_enOrden(raiz);
	}
	void preOrden() {
		_preOrden(raiz);
	}
	void postOrden() {
		_postOrden(raiz);
	}
	int cantidad() {
		return _cantidad(raiz);
	}
	int altura() {
		return _altura(raiz);
	}
    bool eliminar(T e) {
		return _eliminar(raiz, e);
	}
};

#endif


Overwriting ArbolB.hpp


In [23]:
%%writefile Registros.csv
nombre apellido edad decision
Jorge Cerdan 20 Y
Roberto Juarez 35 N
Fidel Sanchez 21 N
Sofia Minchola 34 Y 
Ana Rodriguez 18 N
Juan Perez 36 Y
Jaime Diaz 45 N
Cesar Juarez 17 N
Karla Melendez 38 Y
Agustin Ullon 40 N

Overwriting Registros.csv


In [25]:
%%writefile test.cpp

#include "ArbolB.hpp"
#include <iostream>
#include <string>
#include <functional>
#include <fstream>
#include <ctime>

using namespace std;

class Register {
	string apellido;
	string nombre;
	short edad;
	bool fumador;
public:
	Register(string nombre="", string apellido="", short edad=25, bool fumador=false) {
		this->apellido = apellido;
		this->nombre = nombre;
		this->edad = edad;
		this->fumador = fumador;
	}
	friend ostream& operator<<(ostream& os, const Register& r) {
		os << r.nombre << " " << r.apellido << " " << r.edad << " " << r.fumador << endl;
		return os;
	}
	bool operator<(const Register& r) {
		return this->edad < r.edad;
	}
  bool operator>=(const Register& r) {
		return this->edad >= r.edad;
	}
	bool operator>(const Register& r) {
		return this->edad > r.edad;
	}
};

void imprimir(Register e) {
	cout << " " << e;
}

class Dataset {
	//ArbolB<Register> registros;
    ArbolB<Register>* registros=new ArbolB<Register>(imprimir);
	
public:
	Dataset() {
		readTSV("Registros.csv");
	}
	void readTSV(string name = "", bool header = true) {//campos separados por tab o espacios
		ifstream file(name);
		string reg, nombre, apellido, t_edad, t_fumador;
		short edad;
		bool fumador;
		if (header)
			getline(file, reg);
		while (file >> nombre >> apellido >> t_edad >> t_fumador) {
			edad = stoi(t_edad);
			fumador = t_fumador[0] == 'Y';
			registros->insertar(Register(nombre, apellido, edad, fumador));
		}
	}
		
	void print_enOrden() {
		registros->enOrden();
	}

    void print_postOrden() {
		registros->postOrden();
	}

    void print_preOrden() {
		registros->preOrden();
	}

    int size(){
        return registros->cantidad();
    }

    int Altura(){
        return registros->altura();
    }

    /*void eliminara(){ 
        registros->eliminar(Register("Jorge", "Cerdan", 20, true));
    }*/ //aun falta que funcione
};





int main() {
    Dataset ds;
    cout<<"Ordenamiento enOrden: "<<endl;
    cout<<"----------------------"<<endl;
	  ds.print_enOrden();
    cout<<"\n\nCantidad: "<<ds.size();
    cout<<"\nAltura: "<<ds.Altura();
    cout<<"\n Remove data correct!!"<<endl;

	  cin.get();
  	return 0;
}

Overwriting test.cpp


In [ ]:
!g++ test.cpp && ./a.out

Ordenamiento enOrden: 
----------------------
 Cesar Juarez 17 0
 Ana Rodriguez 18 0
 Jorge Cerdan 20 1
 Fidel Sanchez 21 0
 Sofia Minchola 34 1
 Roberto Juarez 35 0
 Juan Perez 36 1
 Karla Melendez 38 1
 Agustin Ullon 40 0
 Jaime Diaz 45 0


Cantidad: 10
Altura: 6
 Remove data correct!!


##Actividad para la casa 


1.   PreOrden y postOrden   -->listo
2.   altura, Eliminar --->listo, falto un poco de eliminar
3.   Comparacion e imprecion que sean funciones como atributo de clase.-->listo

